<a href="https://colab.research.google.com/github/JoaoVitor733/-DataScience/blob/Master/WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np 

import requests as rq 
from bs4 import BeautifulSoup 
import re  

In [ ]:
link= '/filmes/filme-176535/' #primeira página a ter dados coletados

info_livros = [] #vetor para armazenar todas informações das páginas análisadas
linksUsados = [] #vetor com todos os links usados
linksPodemSerUsados = [] #vetor com os links disponíveis

linksPodemSerUsados.append(link)

for row in linksPodemSerUsados: #andando no vetor dos links
  if len(linksUsados)> 1000: #analisando apenas uma certa quantidade de páginas
      break

  livro = {} #guardará os dados de cada página
  
  try:
    
    listaAux = []
    
    url = 'https://www.adorocinema.com{}'.format(row) #pega o link raiz mais o link da vez
    paginaBruta = rq.get(url) #pega todo texto da página
    paginaParsada = BeautifulSoup(paginaBruta.text ,'html.parser') #cria um objeto capaz de acessar as tags HTML

    livro['Titulo: '] = paginaParsada.find('div',{'class':'titlebar-title titlebar-title-lg'}).get_text() #pega a primeira tag com esse atributo e retorna o texto

    nacionalidade = paginaParsada.find('div',{'class':'item'}).get_text()
    nacionalidade = nacionalidade.split('\n') #retirando os \n presente no texto retornado
    cont = 0
    #tratando o texto com uma lista auxiliar para pegar apenas as nacionalidades
    for x in nacionalidade:
      if not(x == ''):
        if cont > 0:
           listaAux.append(x)
        cont += 1
    livro['Nacionalidade: '] =  str(listaAux)

    divDados = paginaParsada.find('div',{'class':'meta-body'})
    
    livro['Duração: '] = divDados.find('div',{'class':'meta-body-item meta-body-info'}).text.split('/')[1].split('\n')[1] #retirando os / e \n
    
    genero = divDados.find('div',{'class':'meta-body-item meta-body-info'}).text.split('/')[2].split('\n') #retirando os / e \n
    for x in genero: #retirando os espaços em brancos
      if x == '':
        genero.remove(x) 
    livro['Gênero: '] = genero

    livro['Direção: '] =  paginaParsada.find('div',{'class':'meta-body-item meta-body-direction'}).get_text().split('\n')[2]

    livro['Sinopse: '] = paginaParsada.find('div',{'class':'content-txt '}).get_text().replace('\n','').split('.\xa0')[0]#trocando \n por espaço vazio e retirando o .\xa0
    
    secaoFilmes = paginaParsada.find('div',{'class','gd gd-gap-20 gd-xs-2 gd-s-4 last-main-row'})
    link = secaoFilmes.find_all('a',{'class','meta-title-link'})
    for x in range(4):
       testarLink = str(link[x]).split('href="')[1].split('" title=')[0]
       linksPodemSerUsados.append(testarLink) #pegando os 4 links de filmes recomendados de cada página
  
    
    info_livros.append(livro)  #coloca na lista as informações de cada página
    linksUsados.append(link) #armazenando os links ja usados

  except Exception as e:
    print(e)


In [22]:
df = pd.DataFrame(info_livros) #criando um data frame
df.to_csv('./Informacoes.csv') #criando um arquivo csv